# 1. Data processing

## Scrape wikipedia for Toronto neighborhood data

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.text, 'html.parser' )
table_html = soup.find_all('table')



## Convert html to pandas dataframe

In [4]:
#Find table rows
body = table_html[0].find_all('tr')

head = body[0]
body_rows = body[1:]

#Collect table heading
headings = []
for item in head.find_all('th'):
    item = (item.text).rstrip("\n")
    headings.append(item)
    
import re
all_rows = [] # will be a list for list for all rows

for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will hold row entries
    
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        aa = re.sub("(\xa0)|(\n)|","",row_item.text)    
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

neighborhood_df = pd.DataFrame(data=all_rows, columns=["Postal Code", "Borough", 'Neighborhood'])



In [50]:
#Drop 'Not assigned' rows from dataframe and reset index
neighborhood_df.drop(neighborhood_df[neighborhood_df.Borough == 'Not assigned'].index, inplace=True)
neighborhood_df.reset_index(drop=True, inplace=True)

#Sort table by postal code to assist merging of geospace df
sort_df = neighborhood_df.sort_values(['Postal Code']).reset_index(drop=True)
sort_df.head(100)




,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
#Dataframe size
sort_df.shape

(103, 3)

## Read in geospacial csv file

In [7]:
csv_path= '/Users/alexbaylis/Downloads/Geospatial_Coordinates.csv'
geo_df = pd.read_csv(csv_path)

# Only want latitude and longitude info from df
geo2 = geo_df[['Latitude', 'Longitude']]

In [61]:
#Merge the two dataframes
stack_df = pd.concat([sort_df, geo2], axis=1)
stack_df.head(100)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# 2. Explore and Cluster Neighborhoods in Toronto

## Import libraries and packages

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [11]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(stack_df['Latitude'], stack_df['Longitude'], stack_df['Borough'], stack_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [104]:
# Create refined dataframe of central Toronto neighborhoods for clustering later
toronto_data = stack_df[(stack_df['Borough'].isin(['West Toronto', 
                                                   'Central Toronto', 
                                                   'Downtown Toronto', 
                                                   'East Toronto']))].reset_index(drop=True)

toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


## Declare Foursquare Access Tokens

In [12]:
CLIENT_ID = 'OCOCDYUJ25V5B5DUJJCBRQROYERUC1ASPSLSQGZAYDYE4L2' # your Foursquare ID
CLIENT_SECRET = 'SQYE2ZURUKDXYCYST21NLL5AMV4DCGXK2M2FNVJFFGKDUCSR' # your Foursquare Secret
ACCESS_TOKEN = '5YLZILZU1QSFO25QG4XYUO350BH33ZXQPJB0PD0VZUUT5MIO'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OCOCDYUJ25V5B5DUJJCBRQROYERUC1ASPSLSQGZAYDYE4L2
CLIENT_SECRET:SQYE2ZURUKDXYCYST21NLL5AMV4DCGXK2M2FNVJFFGKDUCSR


In [13]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, latitude, longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=OCOCDYUJ25V5B5DUJJCBRQROYERUC1ASPSLSQGZAYDYE4L2&client_secret=SQYE2ZURUKDXYCYST21NLL5AMV4DCGXK2M2FNVJFFGKDUCSR&oauth_token=5YLZILZU1QSFO25QG4XYUO350BH33ZXQPJB0PD0VZUUT5MIO&ll=43.6534817,-79.3839347&v=20180605&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6040dc9defd39343e67dec6c'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 162,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatL

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/alexbaylis/opt/anaconda3/envs/Deadlyrics37/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Poke Guys,Poke Place,43.654895,-79.385052
4,Indigo,Bookstore,43.653515,-79.380696


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Explore Toronto Neighborhoods 


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            ACCESS_TOKEN,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [113]:
toronto_venues = getNearbyVenues(names=stack_df['Neighborhood'],
                                  latitudes=stack_df['Latitude'],
                                  longitudes=stack_df['Longitude'],
                                  radius=500)


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

KeyError: 'groups'

In [131]:
print(toronto_venues.shape)
toronto_venues.head()

(3087, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,NT Home Service Inc.,43.806411,-79.197736,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
4,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service


In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,7,7,7,7,7,7
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Wilson Heights, Downsview North",34,34,34,34,34,34
Bayview Village,6,6,6,6,6,6
"Bedford Park, Lawrence Manor East",53,53,53,53,53,53
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",23,23,23,23,23,23


In [25]:
print('There are {} uniques venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 322 uniques venue categories.


## Analyze Each Neighborhood

First we process the data using one hot encoding.

In [112]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
neighborhood_col = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0, 'Neighborhood', neighborhood_col)


toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie 

In [233]:
toronto_onehot.shape

(3061, 318)

Next, we group rows by neighborhood and take the mean of the frequency of occurence of each category.

In [36]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood']).mean().reset_index()
toronto_grouped


,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie 

In [38]:
toronto_grouped.shape

(98, 322)

Now we print each neighborhood along with the top 5 most common venues.

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
             venue  freq
0  Fireworks Store  0.14
1           Lounge  0.14
2   Breakfast Spot  0.14
3     Skating Rink  0.14
4   Clothing Store  0.14


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.18
1           Gym  0.09
2  Dance Studio  0.09
3          Pool  0.09
4           Pub  0.09


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0  Mobile Phone Shop  0.06
1     Ice Cream Shop  0.06
2               Bank  0.06
3           Pharmacy  0.06
4        Coffee Shop  0.06


----Bayview Village----
                 venue  freq
0                 Café  0.17
1                 Bank  0.17
2                  Spa  0.17
3  Japanese Restaurant  0.17
4                  Gym  0.17


----Bedford Park, Lawrence Manor East----
                venue  freq
0                 Spa  0.09
1         Pizza Place  0.06
2  Italian Restaurant  0.06
3      Massage Studio  0.04
4      Sandwich Place  0.04


----Berczy Park----
             

4     Electronics Store  0.06


----Stn A PO Boxes----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.04
2  Seafood Restaurant  0.04
3        Cocktail Bar  0.04
4                Café  0.03


----Studio District----
         venue  freq
0       Bakery  0.06
1         Café  0.06
2  Coffee Shop  0.06
3  Yoga Studio  0.04
4    Gastropub  0.04


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
                     venue  freq
0              Coffee Shop  0.11
1                     Bank  0.06
2  Health & Beauty Service  0.06
3               Restaurant  0.06
4         Sushi Restaurant  0.06


----The Annex, North Midtown, Yorkville----
                    venue  freq
0                    Café  0.09
1             Coffee Shop  0.09
2          Sandwich Place  0.09
3           Grocery Store  0.06
4  Furniture / Home Store  0.06


----The Beaches----
               venue  freq
0        Pizza Place  0.11
1              Trail  0.11
2  Health 

Lets put this in a pandas dataframe.


In [125]:
#Function to sort venues in descending order:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


#Create dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Fireworks Store,Hardware Store,Skating Rink,Latin American Restaurant,Lounge,Clothing Store,Event Service,Eastern European Restaurant,Electronics Store
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Skating Rink,Pharmacy,Gym,Athletics & Sports,Pub,Sandwich Place,Coffee Shop,Pool
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Mobile Phone Shop,Coffee Shop,Pharmacy,Spa,Ice Cream Shop,Middle Eastern Restaurant,Restaurant,Shopping Mall,Auto Workshop
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Spa,Café,Gym,Yoga Studio,Escape Room,Dumpling Restaurant,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Spa,Italian Restaurant,Pizza Place,Business Service,Juice Bar,Boutique,Coffee Shop,Mobile Phone Shop,Hobby Shop,Sushi Restaurant
5,Berczy Park,Coffee Shop,Sandwich Place,Cocktail Bar,Bakery,Pub,Italian Restaurant,Hotel,Beer Bar,Bistro,Seafood Restaurant
6,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Farm,College Stadium,Café,Yoga Studio,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant
7,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Sandwich Place,Burrito Place,Japanese Restaurant,Bakery,Bar,Stadium
8,"Business reply mail Processing Centre, South C...",Light Rail Station,Gym / Fitness Center,Fast Food Restaurant,Yoga Studio,Brewery,Auto Workshop,Spa,Smoke Shop,Martial Arts School,Skate Park
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Rental Car Location,Airport Lounge,Sculpture Garden,Plane,Boat or Ferry,Coffee Shop,Harbor / Marina,Boutique


# 3. Cluster Neighborhoods
## K-means clustering 

In [348]:
# Its a good idea to scale the features before using k-means clustering
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler_cluster = scaler.fit_transform(toronto_grouped_clustering)


# set number of clusters
kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(scaled_cluster)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:100])

#Silhouette score is a useful metric to understand how many clusters to use: 
# Scores are between -1 and 1.
# Close to  1 indicating instances are well inside their cluster
# Close to 0 means they are on the cluster boundary
# Close to -1 means they are likely in the wrong cluster!  
from sklearn.metrics import silhouette_score
print(f' \n The silhouette score is: {silhouette_score(scaled_cluster, kmeans.labels_)}')

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1]
 
 The silhouette score is: 0.106666582711379


Create a new dataframe that includes the cluster labels as well as the top 10 venues for each neighborhood.

In [349]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Remove NaN row
toronto_merged.dropna(axis=0, how='any', inplace=True)

#Convert float values to int for use in folium
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(np.int64)
toronto_merged.head(100)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Coffee Shop,Park,Home Service,Pizza Place,Asian Restaurant,Pub,Trail,Doner Restaurant,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Sushi Restaurant,Coffee Shop,Ice Cream Shop,Pharmacy,Italian Restaurant,Frozen Yogurt Shop,Bakery,Furniture / Home Store,Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Sandwich Place,Park,Fast Food Restaurant,Pizza Place,Gym,Mobile Phone Shop,Intersection,Pub,Sushi Restaurant,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Bakery,Coffee Shop,Yoga Studio,Music Store,Gastropub,American Restaurant,Furniture / Home Store,Brewery,Middle Eastern Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Construction & Landscaping,Gym / Fitness Center,Swim School,Lawyer,Business Service,Bus Line,Photography Studio,Park,Empanada Restaurant,Drugstore
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Construction & Landscaping,Gym,Convenience Store,Food & Drink Shop,Electronics Store,Sandwich Place,Supplement Shop,Department Store,Dance Studio,Breakfast Spot
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,2,Clothing Store,Spa,Sporting Goods Shop,Salon / Barbershop,Coffee Shop,Cosmetics Shop,Shoe Store,Seafood Restaurant,Restaurant,Café
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Pizza Place,Dessert Shop,Sandwich Place,Gym,Spa,Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,American Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Trail,Park,Tennis Court,Playground,Beer Store,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,2,Coffee Shop,Health & Beauty Service,Aquarium,Liquor Store,Light Rail Station,Spa,Restaurant,Fried Chicken Joint,Supermarket,Sushi Restaurant


Visualise the resulting clusters:

In [350]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 4. Examine the clusters

Here we examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Based on the defining categories, we can assign a name to each cluster.

## Cluster 1

In [351]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,1,Construction & Landscaping,Gym / Fitness Center,Swim School,Lawyer,Business Service,Bus Line,Photography Studio,Park,Empanada Restaurant,Drugstore
22,Central Toronto,1,Home Service,Pool,Health & Beauty Service,Fast Food Restaurant,Ice Cream Shop,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


## Cluster 2

In [352]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Health Food Store,Coffee Shop,Park,Home Service,Pizza Place,Asian Restaurant,Pub,Trail,Doner Restaurant,Donut Shop
1,East Toronto,2,Greek Restaurant,Sushi Restaurant,Coffee Shop,Ice Cream Shop,Pharmacy,Italian Restaurant,Frozen Yogurt Shop,Bakery,Furniture / Home Store,Restaurant
2,East Toronto,2,Sandwich Place,Park,Fast Food Restaurant,Pizza Place,Gym,Mobile Phone Shop,Intersection,Pub,Sushi Restaurant,Food & Drink Shop
3,East Toronto,2,Café,Bakery,Coffee Shop,Yoga Studio,Music Store,Gastropub,American Restaurant,Furniture / Home Store,Brewery,Middle Eastern Restaurant
5,Central Toronto,2,Construction & Landscaping,Gym,Convenience Store,Food & Drink Shop,Electronics Store,Sandwich Place,Supplement Shop,Department Store,Dance Studio,Breakfast Spot
6,Central Toronto,2,Clothing Store,Spa,Sporting Goods Shop,Salon / Barbershop,Coffee Shop,Cosmetics Shop,Shoe Store,Seafood Restaurant,Restaurant,Café
7,Central Toronto,2,Pizza Place,Dessert Shop,Sandwich Place,Gym,Spa,Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,American Restaurant
9,Central Toronto,2,Coffee Shop,Health & Beauty Service,Aquarium,Liquor Store,Light Rail Station,Spa,Restaurant,Fried Chicken Joint,Supermarket,Sushi Restaurant
11,Downtown Toronto,2,Pizza Place,Coffee Shop,Restaurant,Italian Restaurant,Flower Shop,Bakery,Pub,Gourmet Shop,Café,Chinese Restaurant
12,Downtown Toronto,2,Coffee Shop,Sushi Restaurant,Gay Bar,Yoga Studio,Burger Joint,Burrito Place,Japanese Restaurant,Italian Restaurant,Restaurant,Café


## Cluster 3

In [353]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Trail,Park,Tennis Court,Playground,Beer Store,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
10,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Escape Room,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
23,Central Toronto,3,Bus Line,Park,Sushi Restaurant,Trail,Jewelry Store,Health & Beauty Service,Farmers Market,Farm,Donut Shop,Drugstore
